## Termination in Microsoft Autogen

We have used team to do some work, say write a story or accomplish some task. We want them to stop once the task has been completed or if they have run certain number of iterations, otherwise they will keep on going. Therefore autogen has provided us multiple way in which we can Stop or Terminate a teams task.

1. **MaxMessageTermination** : Stop after a specified number of messages have been produced including both agent and task messages.
2. **TextMentionTermination** : Stop when specific text or string is mentioned in a message (e.g, "TERMINATE").
3. **TokenUsageTermination** : Stops when certain number of prompt or completion tokens are used.
4. **TimeoutTermination** : Stops after a specified duration in seconds.
5. **HandoffTermination** : Stops when a handoff to a specific target is requested.
6. **SourcematchTermination** : Stops after a specific agent responds.
7. **ExternalTermination** : Enables programmatic contrl of termination from outside the run.


In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage
from dotenv import load_dotenv
load_dotenv()

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

In [4]:
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message='Add 1 to the number which is given to you as an input and give out result. Start with 0 if othing is provided'
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message='Add 1 to the number provided and pass the result to the next agent'
)

add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message='Add 1 to the number provided and pass result to the next agent'
)

## MaxMessageTermination()

It by defaults stops after certain number of iterations. By default it includes user message as well.

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination

max_message_termination = MaxMessageTermination(5)

team = RoundRobinGroupChat(
    participants=[add_1_agent_first, add_1_agent_second, add_1_agent_third],
    termination_condition=max_message_termination
)

In [8]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task = 'Start from 0'))

---------- TextMessage (user) ----------
Start from 0
---------- TextMessage (add_1_agent_second) ----------
1
---------- TextMessage (add_1_agent_third) ----------
2
---------- TextMessage (add_1_agent_first) ----------
3
---------- TextMessage (add_1_agent_second) ----------
4


TaskResult(messages=[TextMessage(id='458e946a-a2f9-4d0b-b31f-06e160271a46', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 5, 30, 701670, tzinfo=datetime.timezone.utc), content='Start from 0', type='TextMessage'), TextMessage(id='6edff323-6b4d-4ef8-9787-d64f551b3e87', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=100, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 5, 31, 167347, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='49571c89-186e-401f-b0b8-b315e45a3316', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 5, 31, 574795, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='331e69c5-05a3-4f95-a8c7-00fd8ed566a0', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=145, completion_tokens=1), metadata={}, crea

## TextMessageTermination

In [9]:
agent1 = AssistantAgent(
    name='story_writer',
    model_client=model_client,
    system_message='Give the story about a brave knight, keep it short upto 40 words'
)

agent2 = AssistantAgent(
    name='story_critic',
    model_client=model_client,
    system_message="Coninue the story and citic it with feedback. Keep it short and no more than 40 words. If it feels complete, say 'THE END'"
)

In [10]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.messages import TextMessage

text_message_termination = TextMentionTermination("THE END")

team_with_texttermination = RoundRobinGroupChat(
    participants=[agent1, agent2],
    termination_condition=text_message_termination
)

await Console(team_with_texttermination.run_stream(task='Write a story about brave knight'))


---------- TextMessage (user) ----------
Write a story about brave knight
---------- TextMessage (story_writer) ----------
Sir Cedric, renowned for his fearlessness, faced a ferocious dragon threatening his village. Armed with only faith and a sturdy sword, he charged into battle. With a heart full of courage, he defeated the beast, securing peace and honor.
---------- TextMessage (story_critic) ----------
The villagers cheered as Sir Cedric returned, a hero in their eyes. Yet, he humbly reminded them that true bravery lies in protecting the weak. His legend grew, inspiring others to be courageous in their own journeys. 

THE END.


TaskResult(messages=[TextMessage(id='4dff3245-01b3-4a9a-b161-a89411f3c4a6', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 11, 3, 150193, tzinfo=datetime.timezone.utc), content='Write a story about brave knight', type='TextMessage'), TextMessage(id='df0925df-b1e7-43a7-86d0-af768af50ea1', source='story_writer', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=50), metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 11, 4, 397748, tzinfo=datetime.timezone.utc), content='Sir Cedric, renowned for his fearlessness, faced a ferocious dragon threatening his village. Armed with only faith and a sturdy sword, he charged into battle. With a heart full of courage, he defeated the beast, securing peace and honor.', type='TextMessage'), TextMessage(id='c8c84ee2-8128-4b5f-96eb-00257cc11782', source='story_critic', models_usage=RequestUsage(prompt_tokens=107, completion_tokens=50), metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 1

## Combining Termination

We can combine our multiple termination condition with 'AND', 'OR'. If we have combined with OR then team will terminate when any of the condition fullfills.

In [11]:
combined_termination = MaxMessageTermination(3) | TextMentionTermination('THE END')

In [12]:
team_with_combinedTermination = RoundRobinGroupChat(
    participants=[agent1, agent2],
    termination_condition=combined_termination
)

await Console(team_with_texttermination.run_stream(task='Write a story about brave knight'))

---------- TextMessage (user) ----------
Write a story about brave knight
---------- TextMessage (story_writer) ----------
In a kingdom shrouded in darkness, Sir Alaric, a brave knight, learned of a cursed forest where a monstrous creature stole the villagers' hopes. Armed with his gleaming sword and unwavering resolve, he ventured into the shadows. After a fierce battle, he vanquished the beast, freeing the land from despair. The villagers, grateful and filled with hope, celebrated their hero's courage, echoing his bravery for generations to come.
---------- TextMessage (story_critic) ----------
As the festivities unfolded, Sir Alaric shared his belief that courage was a collective strength. Encouraged by his words, the villagers vowed to protect one another, forging a new bond of unity. The knight's legacy inspired a brighter future.

THE END.


TaskResult(messages=[TextMessage(id='5af661c1-6116-4509-a07c-5dee4be3b017', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 17, 2, 41615, tzinfo=datetime.timezone.utc), content='Write a story about brave knight', type='TextMessage'), TextMessage(id='b372eb28-9329-4ea5-a7c3-66db11b2ff40', source='story_writer', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=91), metadata={}, created_at=datetime.datetime(2025, 8, 8, 4, 17, 5, 425625, tzinfo=datetime.timezone.utc), content="In a kingdom shrouded in darkness, Sir Alaric, a brave knight, learned of a cursed forest where a monstrous creature stole the villagers' hopes. Armed with his gleaming sword and unwavering resolve, he ventured into the shadows. After a fierce battle, he vanquished the beast, freeing the land from despair. The villagers, grateful and filled with hope, celebrated their hero's courage, echoing his bravery for generations to come.", type='TextMessage'), TextMessag

### ExternalTermination()

Enables programmatic control of termination from outside the run. This is useful for UIintegration (e.g, "STOP" buton in chat interface).

In [13]:
from autogen_agentchat.conditions import ExternalTermination

external_termination = ExternalTermination()

In [14]:
team_with_externaltermination = RoundRobinGroupChat(
    participants=[agent1, agent2],
    termination_condition=external_termination
)

run = asyncio.create_task(Console(team_with_externaltermination.run_stream(task ='Write a story about brave knight')))

await asyncio.sleep(2)

## terminating the team by setting the externaltermination
external_termination.set()

await run

---------- TextMessage (user) ----------
Write a story about brave knight
---------- TextMessage (story_writer) ----------
In a realm beset by shadows, Sir Roland, a courageous knight, vowed to protect his castle from the malevolent sorceress threatening to engulf the land in darkness. With his loyal steed and enchanted sword, he journeyed through treacherous terrains. 

As dusk fell, he confronted the sorceress atop a cliff, their battle fierce and relentless. Sir Roland, fueled by the cries of his people, summoned all his strength, ultimately breaking her curse with a radiant strike of his sword. 

Peace restored, the kingdom flourished, and Sir Roland became a legend, reminding all that true bravery shines brightest in the face of fear.


TaskResult(messages=[TextMessage(id='1b470aba-7d89-43e7-8588-4e36eecc850b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 2, 47, 419185, tzinfo=datetime.timezone.utc), content='Write a story about brave knight', type='TextMessage'), TextMessage(id='f3f7939f-b1fb-42f4-b0fe-9258235dafa0', source='story_writer', models_usage=RequestUsage(prompt_tokens=262, completion_tokens=131), metadata={}, created_at=datetime.datetime(2025, 8, 8, 5, 2, 50, 762790, tzinfo=datetime.timezone.utc), content='In a realm beset by shadows, Sir Roland, a courageous knight, vowed to protect his castle from the malevolent sorceress threatening to engulf the land in darkness. With his loyal steed and enchanted sword, he journeyed through treacherous terrains. \n\nAs dusk fell, he confronted the sorceress atop a cliff, their battle fierce and relentless. Sir Roland, fueled by the cries of his people, summoned all his strength, ultimately breaking her curse with a radiant

#### So the team will not stop immediately instead it complete the running agent task and then will terminate.